In [1]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [48]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from pprint import pprint
# from konlpy.tag import Mecab
from nltk.tokenize import word_tokenize as en_tokenizer
import sentencepiece as spm
import urllib.request
import csv
import numpy as np
from einops import rearrange, reduce, repeat
from torch.cuda import amp
from tqdm import tqdm
import wandb
import time
import copy
from collections import defaultdict
from sklearn.metrics import mean_squared_error
import joblib
import gc
import os
from icecream import ic
# 어느 코드에서 진행된 것인지
# debug할 때 편리함을 줌!
from sklearn.model_selection import train_test_split
import os

In [92]:
VOCAB_SIZE = 5000
SEQ_LEN = 60


PAD_IDX = 0
BOS_IDX = 2
EOS_IDX = 3


# ENV = 'COLAB'
ENV = 'KAGGLE'
# ENV = 'SYSTEM'

# Option for Mixed Precision
FP16 = True
# FP16 = False

N = 2
HIDDEN_DIM = 256
NUM_HEAD = 8 
INNER_DIM = 512
BATCH_SIZE = 64
LEARNING_RATE = 1e-4
WEIGHT_DECAY = 0


CONFIG = {
    'VOCAB_SIZE': VOCAB_SIZE,
    'SEQ_LEN': SEQ_LEN,
    'N': N,
    'HIDDEN_DIM': HIDDEN_DIM,
    'NUM_HEAD': NUM_HEAD,
    'INNER_DIM': INNER_DIM,
    'BATCH_SIZE': BATCH_SIZE,
    'WEIGHT_DECAY' : WEIGHT_DECAY,
    'LEARNING_RATE' : LEARNING_RATE,
}


if 'device' not in globals():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using {device}')

Using cpu


Data load에선 문제가 생겨 추후 문제점 해결하려고 함 (wandb에 github id를 활용하여 id 생성 후 연결하는 작업 진행)

In [50]:
'''
import wandb
import os
# if want to run in offline mode

# os.environ["WANDB_MODE"] = "dryrun"
# wandb.init(project="Transformer_bible", entity="jiwon7258")
os.environ["WANDB_MODE"] = "online"

wandb.init(project="Transformer_bible", entity="thomas7225", config = CONFIG, job_type = 'train')
wandb.run.name = f"train_{VOCAB_SIZE}_{SEQ_LEN}_{N}_{HIDDEN_DIM}_{INNER_DIM}"


dataset = wandb.Artifact(f'bible-dataset_{VOCAB_SIZE}_{SEQ_LEN}', type='dataset')4
'''

'\nimport wandb\nimport os\n# if want to run in offline mode\n\n# os.environ["WANDB_MODE"] = "dryrun"\n# wandb.init(project="Transformer_bible", entity="jiwon7258")\nos.environ["WANDB_MODE"] = "online"\n\nwandb.init(project="Transformer_bible", entity="thomas7225", config = CONFIG, job_type = \'train\')\nwandb.run.name = f"train_{VOCAB_SIZE}_{SEQ_LEN}_{N}_{HIDDEN_DIM}_{INNER_DIM}"\n\n\ndataset = wandb.Artifact(f\'bible-dataset_{VOCAB_SIZE}_{SEQ_LEN}\', type=\'dataset\')4\n'

### Load

In [51]:
'''
dataset = wandb.run.use_artifact(f'bible-dataset_{VOCAB_SIZE}_{SEQ_LEN}:latest')

# Download the artifact's contents
artifact_dir = dataset.download()
'''

"\ndataset = wandb.run.use_artifact(f'bible-dataset_{VOCAB_SIZE}_{SEQ_LEN}:latest')\n\n# Download the artifact's contents\nartifact_dir = dataset.download()\n"

In [52]:
'''
# or Train / Valid Data
src_train_path = os.path.join(artifact_dir,'src_train.pkl')
src_valid_path = os.path.join(artifact_dir,'src_valid.pkl')
trg_train_path = os.path.join(artifact_dir,'trg_train.pkl')
trg_valid_path = os.path.join(artifact_dir,'trg_valid.pkl')

src_train = joblib.load(src_train_path)
src_valid = joblib.load(src_valid_path)
trg_train = joblib.load(trg_train_path)
trg_valid = joblib.load(trg_valid_path)
'''

"\n# or Train / Valid Data\nsrc_train_path = os.path.join(artifact_dir,'src_train.pkl')\nsrc_valid_path = os.path.join(artifact_dir,'src_valid.pkl')\ntrg_train_path = os.path.join(artifact_dir,'trg_train.pkl')\ntrg_valid_path = os.path.join(artifact_dir,'trg_valid.pkl')\n\nsrc_train = joblib.load(src_train_path)\nsrc_valid = joblib.load(src_valid_path)\ntrg_train = joblib.load(trg_train_path)\ntrg_valid = joblib.load(trg_valid_path)\n"

### 새로운 방법론 (데이터셋 직접다운)

In [53]:
import os

DATASET_PATH = './data'

# 파일을 열 때 인코딩 방식을 UTF-8로 지정
with open(os.path.join(DATASET_PATH, 'bible-all.en.txt'), encoding='utf-8') as en_train:
    en_train_content = en_train.read()
en_train_list = en_train_content.split('\n')

with open(os.path.join(DATASET_PATH, 'bible-all.kr.txt'), encoding='utf-8') as ko_train:
    ko_train_content = ko_train.read()
ko_train_list = ko_train_content.split('\n')

# 첫 10개 항목을 출력
print(en_train_list[:10])


['Genesis1.1  In the beginning God created the heavens and the earth.', 'Genesis1.2  Now the earth was formless and empty, darkness was over the surface of the deep, and the Spirit of God was hovering over the waters.', 'Genesis1.3  And God said, "Let there be light," and there was light.', 'Genesis1.4  God saw that the light was good, and He separated the light from the darkness.', 'Genesis1.5  God called the light "day," and the darkness he called "night." And there was evening, and there was morning--the first day.', 'Genesis1.6  And God said, "Let there be an expanse between the waters to separate water from water."', 'Genesis1.7  So God made the expanse and separated the water under the expanse from the water above it. And it was so.', 'Genesis1.8  God called the expanse "sky." And there was evening, and there was morning--the second day.', 'Genesis1.9  And God said, "Let the water under the sky be gathered to one place, and let dry ground appear." And it was so.', 'Genesis1.10  G

In [54]:
data = pd.DataFrame()
data['en_raw'] = en_train_list
data['ko_raw'] = ko_train_list
data = data.reset_index(drop = True)
print(len(data))
data.head()

31104


,en_raw,ko_raw
0,Genesis1.1 In the beginning God created the h...,Genesis1.1 태초에 하나님이 천지를 창조하셨다.
1,Genesis1.2 Now the earth was formless and emp...,"Genesis1.2 땅이 혼돈하고 공허하며, 어둠이 깊음 위에 있고, 하나님의 영..."
2,"Genesis1.3 And God said, ""Let there be light,...","Genesis1.3 하나님이 말씀하시기를 ""빛이 생겨라"" 하시니, 빛이 생겼다."
3,"Genesis1.4 God saw that the light was good, a...","Genesis1.4 그 빛이 하나님 보시기에 좋았다. 하나님이 빛과 어둠을 나누셔서,"
4,"Genesis1.5 God called the light ""day,"" and th...","Genesis1.5 빛을 낮이라고 하시고, 어둠을 밤이라고 하셨다. 저녁이 되고 ..."


In [55]:
data['en'] = data['en_raw'].apply(lambda x: x.split(' ')[1:])
data['en'] = data['en'].apply(lambda x: (' ').join(x))
data['ko'] = data['ko_raw'].apply(lambda x: x.split(' ')[1:])
data['ko'] = data['ko'].apply(lambda x: (' ').join(x))

In [56]:
data = data[['en','ko']]
data.head()

,en,ko
0,In the beginning God created the heavens and ...,태초에 하나님이 천지를 창조하셨다.
1,"Now the earth was formless and empty, darknes...","땅이 혼돈하고 공허하며, 어둠이 깊음 위에 있고, 하나님의 영은 물 위에 움직이고..."
2,"And God said, ""Let there be light,"" and there...","하나님이 말씀하시기를 ""빛이 생겨라"" 하시니, 빛이 생겼다."
3,"God saw that the light was good, and He separ...","그 빛이 하나님 보시기에 좋았다. 하나님이 빛과 어둠을 나누셔서,"
4,"God called the light ""day,"" and the darkness ...","빛을 낮이라고 하시고, 어둠을 밤이라고 하셨다. 저녁이 되고 아침이 되니, 하루가..."


### 단어 사전 만들기

In [57]:
with open('src.txt', mode = 'w', encoding='utf8') as f:
    f.write('\n'.join(data['en']))
with open('trg.txt', mode= 'w', encoding='utf8') as f:
    f.write('\n'.join(data['ko']))

In [58]:
%%capture context
corpus = "src.txt"
prefix = "src"
spm.SentencePieceTrainer.train(
    f"--input={corpus} --model_prefix={prefix} --vocab_size={VOCAB_SIZE}" +
    " --model_type=bpe" +
    " --max_sentence_length=999999" +  # 문장 최대 길이
    " --pad_id=0 --pad_piece=[PAD]" +  # pad (0)
    " --unk_id=1 --unk_piece=[UNK]" +  # unknown (1)
    " --bos_id=2 --bos_piece=[BOS]" +  # begin of sequence (2)
    " --eos_id=3 --eos_piece=[EOS]" +  # end of sequence (3)
    " --user_defined_symbols=[SEP],[CLS],[MASK]");  # 사용자 정의 토큰

In [59]:
%%capture context
corpus = "trg.txt"
prefix = "trg"
spm.SentencePieceTrainer.train(
    f"--input={corpus} --model_prefix={prefix} --vocab_size={VOCAB_SIZE}" +
    " --model_type=bpe" +
    " --max_sentence_length=999999" +  # 문장 최대 길이
    " --pad_id=0 --pad_piece=[PAD]" +  # pad (0)
    " --unk_id=1 --unk_piece=[UNK]" +  # unknown (1)
    " --bos_id=2 --bos_piece=[BOS]" +  # begin of sequence (2)
    " --eos_id=3 --eos_piece=[EOS]" +  # end of sequence (3)
    " --user_defined_symbols=[SEP],[CLS],[MASK]");  # 사용자 정의 토큰

#### 정수 인코딩

In [60]:
sp_src = spm.SentencePieceProcessor()
sp_src.Load('src.model')


for idx in range(3):
    sentence = data['en'][idx]
    print(sp_src.EncodeAsPieces(sentence))
    print(sp_src.EncodeAsIds(sentence))

['▁In', '▁the', '▁beginning', '▁God', '▁created', '▁the', '▁heavens', '▁and', '▁the', '▁earth', '.']
[502, 10, 2155, 133, 3212, 10, 1354, 19, 10, 458, 4961]
['▁Now', '▁the', '▁earth', '▁was', '▁form', 'less', '▁and', '▁empty', ',', '▁darkness', '▁was', '▁over', '▁the', '▁sur', 'f', 'ace', '▁of', '▁the', '▁deep', ',', '▁and', '▁the', '▁Spirit', '▁of', '▁God', '▁was', '▁ho', 'ver', 'ing', '▁over', '▁the', '▁waters', '.']
[589, 10, 458, 127, 3464, 636, 19, 3330, 4958, 1451, 127, 268, 10, 1018, 4953, 231, 21, 10, 1685, 4958, 19, 10, 837, 21, 133, 127, 386, 94, 39, 268, 10, 1411, 4961]
['▁And', '▁God', '▁said', ',', '▁"', 'Let', '▁there', '▁be', '▁light', ',"', '▁and', '▁there', '▁was', '▁light', '.']
[288, 133, 150, 4958, 65, 1612, 250, 52, 897, 393, 19, 250, 127, 897, 4961]


In [61]:
def en_encode(tmpstr:str) -> np.array :
    tmpstr = np.array(sp_src.EncodeAsIds(tmpstr))

    # SEQ_LEN보다 길면 짜른다 
    if len(tmpstr) > SEQ_LEN :
        tmpstr = tmpstr[:SEQ_LEN]

    # SEQ_LEN보다 작으면 padding
    else :
        tmpstr = np.pad(tmpstr, (0, SEQ_LEN - len(tmpstr)), 'constant', constant_values = sp_src.pad_id())
    
    return tmpstr

In [62]:
# src_data는 data['en']를 참조한다. (동일 id)
src_data = data['en']

src_list = []

for idx in range(len(src_data)):
    src_list.append(en_encode(src_data[idx]))

src_list[0]

array([ 502,   10, 2155,  133, 3212,   10, 1354,   19,   10,  458, 4961,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0])

In [63]:
sp_trg = spm.SentencePieceProcessor()
sp_trg.Load('trg.model')


for idx in range(3):
    sentence = data['ko'][idx]
    print(sp_trg.EncodeAsPieces(sentence))
    print(sp_trg.EncodeAsIds(sentence))

['▁태', '초', '에', '▁하나님이', '▁천', '지를', '▁창조', '하셨다', '.']
[561, 4349, 3964, 213, 369, 513, 2208, 883, 3962]
['▁땅이', '▁혼', '돈', '하고', '▁공', '허', '하며', ',', '▁어둠', '이', '▁깊', '음', '▁위에', '▁있고', ',', '▁하나님의', '▁영', '은', '▁물', '▁위에', '▁움', '직', '이고', '▁계', '셨다', '.']
[1226, 1567, 4398, 106, 440, 4291, 455, 3961, 1710, 3959, 1114, 4043, 394, 696, 3961, 194, 153, 3978, 119, 394, 2892, 4226, 411, 193, 91, 3962]
['▁하나님이', '▁말씀하시기를', '▁"', '빛', '이', '▁생', '겨', '라', '"', '▁하시니', ',', '▁빛이', '▁생', '겼다', '.']
[213, 2045, 32, 4386, 3959, 171, 4156, 3983, 3995, 2921, 3961, 3057, 171, 1450, 3962]


In [64]:
def ko_encode(tmpstr: str) -> np.array:
    tmpstr = np.array(sp_trg.EncodeAsIds(tmpstr))
    tmpstr = np.insert(tmpstr, 0, sp_trg.bos_id())

    if len(tmpstr) >= SEQ_LEN:
        # SEQ_LEN -1의 길이로 자른다
        tmpstr = tmpstr[:SEQ_LEN-1]
        # 마지막에 <eos> 토큰을 넣어줌으로써, 길이를 SEQ_LEN으로 맞춘다
        tmpstr = np.pad(tmpstr, (0, 1),
                        'constant', constant_values=sp_trg.eos_id())


    else:
        tmpstr = np.pad(tmpstr, (0, 1),
                        'constant', constant_values=sp_trg.eos_id())
        tmpstr = np.pad(tmpstr, (0, SEQ_LEN - len(tmpstr)),
                        'constant', constant_values=sp_trg.pad_id())

    return tmpstr

In [65]:
# trg_data는 data['ko']를 참조한다. (동일 id)
trg_data = data['ko']

trg_list = []

for idx in range(len(trg_data)):
    trg_list.append(ko_encode(trg_data[idx]))   

trg_list[0]

array([   2,  561, 4349, 3964,  213,  369,  513, 2208,  883, 3962,    3,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0])

In [66]:
src_train, src_valid, trg_train, trg_valid = train_test_split(src_list, trg_list, test_size=0.2, random_state=42)


In [67]:
class TrainDataset(Dataset):
    def __init__(self, src_data, trg_data):
        super().__init__()

        assert len(src_data) == len(trg_data)

        self.src_data = src_data
        self.trg_data = trg_data

    def __len__(self):
        return len(self.src_data)
        
    def __getitem__ (self, idx):
        src = self.src_data[idx]
        trg_input = self.trg_data[idx]
        trg_output = trg_input[1:SEQ_LEN]
        trg_output = np.pad(trg_output, (0,1), 'constant', constant_values =0)
        # (seq_len,)
        return torch.Tensor(src).long(), torch.Tensor(trg_input).long(), torch.Tensor(trg_output).long()

train_dataset = TrainDataset(src_train, trg_train)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle= True, pin_memory=True)




In [68]:
class ValidDataset(Dataset):
    def __init__(self, src_data, trg_data):
        super().__init__()

        assert len(src_data) == len(trg_data)

        self.src_data = src_data
        self.trg_data = trg_data

    def __len__(self):
        return len(self.src_data)
        
    def __getitem__ (self, idx):
        src = self.src_data[idx]
        trg_input = self.trg_data[idx]
        trg_output = trg_input[1:SEQ_LEN]
        trg_output = np.pad(trg_output, (0,1), 'constant',constant_values= 0)

        return torch.Tensor(src).long(), torch.Tensor(trg_input).long(), torch.Tensor(trg_output).long()

valid_dataset = ValidDataset(src_valid, trg_valid)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle= False, pin_memory=True)




### Mask Function

In [69]:
# Mask 행렬을 반환하는 Mask Function
# Masking은 QK_T 중 srcK 의 seq_len을 중심으로 한다는 점을 알아두자!!
'''
Input
- Tensor
    shape (bs, srcK seq_len)
Args
- Option
    If option is 'padding', function returns padding mask
    If option is 'lookahead', function returns lookahead mask
Output
- Tensor (option = 'padding' )
    shape (bs, 1, 1, srcK seq_len)
    
* shape 중 (1, 1) 부분은 broad casting을 위한 것이다.
'''
def makeMask(tensor, option: str) -> torch.Tensor:
    '''
    tensor (bs, seq_len)
    '''
    if option == 'padding':
        tmp = torch.full_like(tensor, fill_value=PAD_IDX).to(device)
        # tmp : (bs,seq_len)
        mask = (tensor != tmp).float()
        # mask : (bs, seq_len)
        mask = rearrange(mask, 'bs seq_len -> bs 1 1 seq_len ')
        
        # mask(bs, 1, seq_len,seq_len)

        '''
        Example of mask
        tensor([[
         [1., 1., 1., 1., 0., 0., 0., 0.]]])
        '''

    elif option == 'lookahead':
        # srcQ의 seq_len과 srcK의 seq_len이 동일하다고 가정한다
        # tensor : (bs, seq_len)

        padding_mask = makeMask(tensor, 'padding')
        padding_mask = repeat(
            padding_mask, 'bs 1 1 k_len -> bs 1 new k_len', new=padding_mask.shape[3])
        # padding_mask : (bs, 1, seq_len, seq_len)

        '''
        Example of padding_mask
        tensor([[
         [1., 1., 1., 1., 0., 0., 0., 0.]
         [1., 1., 1., 1., 0., 0., 0., 0.]
         [1., 1., 1., 1., 0., 0., 0., 0.]
         [1., 1., 1., 1., 0., 0., 0., 0.]
         [1., 1., 1., 1., 0., 0., 0., 0.]
         [1., 1., 1., 1., 0., 0., 0., 0.]
         [1., 1., 1., 1., 0., 0., 0., 0.]
         [1., 1., 1., 1., 0., 0., 0., 0.]]])
        '''
        mask = torch.ones_like(padding_mask)
        mask = torch.tril(mask)

        '''
        Example of 'mask'
        tensor([[
        [1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]]])
        '''

        mask = mask * padding_mask
        # ic(mask.shape)

        '''
        Example
        tensor([[
         [1., 0., 0., 0., 0., 0., 0., 0.],
         [1., 1., 0., 0., 0., 0., 0., 0.],
         [1., 1., 1., 0., 0., 0., 0., 0.],
         [1., 1., 1., 1., 0., 0., 0., 0.],
         [1., 1., 1., 1., 0., 0., 0., 0.],
         [1., 1., 1., 1., 0., 0., 0., 0.],
         [1., 1., 1., 1., 0., 0., 0., 0.],
         [1., 1., 1., 1., 0., 0., 0., 0.]]])
        '''

    return mask

- test code를 통해 이해

In [70]:
test = torch.Tensor([[1,2,3,4,5,6,0,0,0,0]])
ic(test.shape)
# option padding => 0이 아닌 값에 대해서 1의 값을 줌
test1 = makeMask(test, option = 'padding')
# option lookahead 
# repeat => 차원을 확장시킴 ex) 10X1 => 10X10
# ones_like => 해당 행렬을 하삼각행렬의 형태로 만들어줌
# mask를 통해 masking 진행
# masking을 통해 미래 정보를 반영하지 않게 함
test2 = makeMask(test, option = 'lookahead')
ic(test1.shape)
print()
print(" padding  옵션에 대한 출력 \n =>", test1)
print()
ic(test2.shape)
print("lookahead 옵션에 대한 출력 \n =>", test2)



 padding  옵션에 대한 출력 
 => tensor([[[[1., 1., 1., 1., 1., 1., 0., 0., 0., 0.]]]])

lookahead 옵션에 대한 출력 
 => tensor([[[[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
          [1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
          [1., 1., 1., 1., 0., 0., 0., 0., 0., 0.],
          [1., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
          [1., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
          [1., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
          [1., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
          [1., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
          [1., 1., 1., 1., 1., 1., 0., 0., 0., 0.]]]])


### Multihead Self Attention

In [71]:
class Multiheadattention(nn.Module):
    def __init__(self, hidden_dim: int, num_head: int):
        super().__init__()

        # embedding_dim, d_model, 512 in paper
        self.hidden_dim = hidden_dim
        # 8 in paper
        self.num_head = num_head
        # head_dim, d_key, d_query, d_value, 64 in paper (= 512 / 8)
        self.head_dim = hidden_dim // num_head
        self.scale = torch.sqrt(torch.FloatTensor()).to(device)

        
        # query, key, value 값에 대해 선형변환을 진행해준다.
        # 새로운 차원(데이터)으로 매핑시켜준다 (단, 해당코드에서는 차원이 변하지 않음)
        
        self.fcQ = nn.Linear(hidden_dim, hidden_dim)
        self.fcK = nn.Linear(hidden_dim, hidden_dim)
        self.fcV = nn.Linear(hidden_dim, hidden_dim)
        self.fcOut = nn.Linear(hidden_dim, hidden_dim)
        #dropout으로 과적합 방지
        self.dropout = nn.Dropout(0.1)

    # 순전파 과정
    def forward(self, srcQ, srcK, srcV, mask=None):

        ##### SCALED DOT PRODUCT ATTENTION ######
        # 
        # input : (bs(= batchsize), seq_len, hidden_dim)
        Q = self.fcQ(srcQ)
        K = self.fcK(srcK)
        V = self.fcV(srcV)
        # 텐서의 차원을 변환하고 재배치하는 역할 
        # 주어진 텐서의 차원을 재구성하여 원하는 형태로 바꾸는 데 사용
        # ()안에 있다면 값을 곱해주는 것임
        # num_head => mutli head의 개수
        # seq_len => sequence length (문장의 갯수)
        # head_dim => embedding 시킨 input 차원
        Q = rearrange(
            Q, 'bs seq_len (num_head head_dim) -> bs num_head seq_len head_dim', num_head=self.num_head)
        K_T = rearrange(
            K, 'bs seq_len (num_head head_dim) -> bs num_head head_dim seq_len', num_head=self.num_head)
        V = rearrange(
            V, 'bs seq_len (num_head head_dim) -> bs num_head seq_len head_dim', num_head=self.num_head)

        attention_energy = torch.matmul(Q, K_T)
        # attention_energy : (bs, num_head, q_len, k_len)

        if mask is not None :
            '''
            mask.shape
            if padding : (bs, 1, 1, k_len)
            if lookahead : (bs, 1, q_len, k_len)
            '''
            # masking 해야될 곳에 큰 음수의 값을 지정해준다
            attention_energy = torch.masked_fill(attention_energy, (mask == 0), -1e+4)
        
        # softmax를 활용을 통해 확률분포로 변환하는 과정
        attention_energy = torch.softmax(attention_energy, dim = -1)

        result = torch.matmul(self.dropout(attention_energy),V)
        # result (bs, num_head, seq_len, head_dim)

        ##### END OF SCALED DOT PRODUCT ATTENTION ######

        # CONCAT
        result = rearrange(result, 'bs num_head seq_len head_dim -> bs seq_len (num_head head_dim)')
        # result : (bs, seq_len, hidden_dim)

        # LINEAR

        result = self.fcOut(result)

        return result

In [72]:
# TEST CODE #
bs = 1
hidden_dim = 10
src = torch.Tensor([[1,2,3,4,5,6,0,0,0,0]])
ic(src.shape)
padding_mask = makeMask(src, option = 'padding')
ic(padding_mask.shape)
lookahead_mask = makeMask(src, option = 'lookahead')
ic(lookahead_mask.shape)


test_Q = torch.randn((bs,2,10))
test_K = rearrange(src, 'seq_len hidden_dim -> 1 seq_len hidden_dim')
ic(test_K.shape)
test_K = repeat(src, 'seq_len hidden_dim -> 1 (10 seq_len) hidden_dim')
ic(test_Q.shape)
ic(test_K.shape)
test_layer = Multiheadattention(hidden_dim=hidden_dim, num_head =2)
ic(test_layer(srcQ = test_Q, srcK = test_K, srcV = test_K, mask = padding_mask).shape)




torch.Size([1, 2, 10])

### Positionwise Feedforward Network

In [73]:
class FFN(nn.Module):
    def __init__ (self, hidden_dim, inner_dim):
        super().__init__()

        # 512 in paper 
        self.hidden_dim = hidden_dim
        # 2048 in paper
        self.inner_dim = inner_dim 

        self.fc1 = nn.Linear(hidden_dim, inner_dim)
        self.fc2 = nn.Linear(inner_dim, hidden_dim)
        self.relu = nn.ReLU(inplace=False)
        self.dropout = nn.Dropout(0.1)


        
    def forward(self, input):
        output = input
        output = self.fc1(output)
        output2 = self.relu(output)
        output2 = self.dropout(output)
        output3 = self.fc2(output2)

        return output3

### Encoder layer

In [74]:
class EncoderLayer(nn.Module):
    def __init__(self, hidden_dim, num_head, inner_dim):
        super().__init__()

        self.hidden_dim = hidden_dim
        self.num_head = num_head
        self.inner_dim = inner_dim
        
        self.multiheadattention = Multiheadattention(hidden_dim, num_head)
        self.ffn = FFN(hidden_dim, inner_dim)
        #layernorm -> layer 정규화를 진행
        self.layerNorm1 = nn.LayerNorm(hidden_dim)
        self.layerNorm2 = nn.LayerNorm(hidden_dim)


        self.dropout1 = nn.Dropout(p=0.1)
        self.dropout2 = nn.Dropout(p=0.1)


    def forward(self, input, mask = None):

        # input : (bs, seq_len, hidden_dim)
        
        # 드롭아웃 및 잔차연결
        output = self.multiheadattention(srcQ= input, srcK = input, srcV = input, mask = mask)
        output = self.dropout1(output)
        output = input + output
        output = self.layerNorm1(output)

        output_ = self.ffn(output)
        output_ = self.dropout2(output_)
        output = output + output_
        output = self.layerNorm2(output)

        # output : (bs, seq_len, hidden_dim)
        return output

### Encoder Architecture

In [75]:
class Encoder(nn.Module):
    def __init__ (self, N, hidden_dim, num_head, inner_dim,max_length=100):
        super().__init__()

        # N : encoder layer의 층
        self.N = N
        self.hidden_dim = hidden_dim
        self.num_head = num_head
        self.inner_dim = inner_dim

        # embedding 해줌
        # nn.ModuleList -> module의 List를 관리
        self.embedding = nn.Embedding(num_embeddings=VOCAB_SIZE, embedding_dim=hidden_dim, padding_idx=0)
        self.pos_embedding = nn.Embedding(max_length, hidden_dim)
        self.enc_layers = nn.ModuleList([EncoderLayer(hidden_dim, num_head, inner_dim) for _ in range(N)])

        self.dropout = nn.Dropout(p=0.1)



    def forward(self, input):
        
        batch_size = input.shape[0]
        seq_len = input.shape[1]
        # input : (bs, seq_len)

        mask = makeMask(input, option='padding')

        #arrange() => ()안에 값들을 순서대로 나열해줌 
        # pos => 위치 인덱스를 생성(각 시퀀스 길이에 대해)
        pos = torch.arange(0, seq_len).unsqueeze(0).repeat(batch_size, 1).to(device)
        # pos: [batch_size, src_len]

        # embedding layer
        output = self.dropout(self.embedding(input) + self.pos_embedding(pos))
        # output : (bs, seq_len, hidden_dim)


        # Positional Embedding
        # output = pos_embed(output)

        # Dropout
        output = self.dropout(output)

        # N개의 layer를 순차적으로 돌게 만들어줌
        for layer in self.enc_layers:
            output = layer(output, mask)

        # output : (bs, seq_len, hidden_dim)

        return output

### Decoder Layer

In [76]:
class DecoderLayer(nn.Module):
    def __init__(self, hidden_dim, num_head, inner_dim):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.num_head = num_head
        self.inner_dim = inner_dim

        
        # 2개의 multiheadattention
        # 1개의 FeedForward Network
        self.multiheadattention1 = Multiheadattention(hidden_dim, num_head)
        self.layerNorm1 = nn.LayerNorm(hidden_dim)
        self.multiheadattention2 = Multiheadattention(hidden_dim, num_head)
        self.layerNorm2 = nn.LayerNorm(hidden_dim)
        self.ffn = FFN(hidden_dim, inner_dim)
        self.layerNorm3 = nn.LayerNorm(hidden_dim)

        self.dropout1 = nn.Dropout(p=0.1)
        self.dropout2 = nn.Dropout(p=0.1)
        self.dropout3 = nn.Dropout(p=0.1)

    
    def forward(self, input, enc_output, paddingMask, lookaheadMask):
        # input : (bs, seq_len, hidden_dim)
        # enc_output : (bs, seq_len, hidden_dim)

        # 1번째 multiheadattention
        # 드롭아웃 및 잔차연결
        output = self.multiheadattention1(input, input, input, lookaheadMask)
        output = self.dropout1(output)
        output = output + input
        output = self.layerNorm1(output)


        # 2번째 multiheadattention
        # 드롭아웃 및 잔차연결
        output_ = self.multiheadattention2(output, enc_output, enc_output, paddingMask)
        output_ = self.dropout2(output_)
        output = output_ + output
        output = self.layerNorm2(output)



        # Feedforward Network
        # 드롭아웃 및 잔차연결
        output_ = self.ffn(output)
        output_ = self.dropout3(output_)
        output = output + output_
        output = self.layerNorm3(output)



        return output


### Decoder Architecture

In [77]:
class Decoder(nn.Module):
    def __init__ (self, N, hidden_dim, num_head, inner_dim, max_length=100):
        super().__init__()

        # N : number of encoder layer repeated 
        self.N = N
        self.hidden_dim = hidden_dim
        self.num_head = num_head
        self.inner_dim = inner_dim

        # embedding layer 설정
        self.embedding = nn.Embedding(num_embeddings=VOCAB_SIZE, embedding_dim=hidden_dim, padding_idx=0)
        self.pos_embedding = nn.Embedding(max_length, hidden_dim)

        # decoder N개 생성
        self.dec_layers = nn.ModuleList([DecoderLayer(hidden_dim, num_head, inner_dim) for _ in range(N)])

        self.dropout = nn.Dropout(p=0.1)
        
        #최종 선형변환에 필요한 Linear작업
        self.finalFc = nn.Linear(hidden_dim, VOCAB_SIZE)


    def forward(self, input, enc_src, enc_output):

        # input = dec_src : (bs, seq_len)
        # enc_src : (bs, seq_len)
        # enc_output : (bs, seq_len,hidden_dim)
        
        lookaheadMask = makeMask(input, option= 'lookahead')
        paddingMask = makeMask(enc_src, option = 'padding')

        # embedding layer
        output = self.embedding(input)
        # output = (bs, seq_len, hidden_dim)


        # Positional Embedding
        # output = pos_embed(output)

        # Dropout
        output = self.dropout(output)

        # N decoder layer
        for layer in self.dec_layers:
            output = layer(output, enc_output, paddingMask, lookaheadMask)
        # output : (bs, seq_len, hidden_dim)

        # 최종값이기 때문에 for문 밖에서 진행해주어야됨
        # 해당 vocab에 맞게 차원을 변환
        logits = self.finalFc(output)
        # logits : (bs, seq_len, VOCAB_SIZE)
        output = torch.softmax(logits, dim = -1)

        output = torch.argmax(output, dim = -1)
        # output : (bs, seq_len), dtype=int64



        return logits, output

### Transformer Model

In [78]:
class Transformer(nn.Module):
    def __init__(self, N = 2, hidden_dim = 256, num_head = 8, inner_dim = 512):
        super().__init__()
        self.encoder = Encoder(N, hidden_dim, num_head, inner_dim)
        self.decoder = Decoder(N, hidden_dim, num_head, inner_dim)

    def forward(self, enc_src, dec_src):
        # enc_src : (bs, seq_len)
        # dec_src : (bs, seq_len)

        # print(f'enc_src : {enc_src.shape}')
        # print(f'dec_src : {dec_src.shape}')

        enc_output = self.encoder(enc_src)
        # enc_output : (bs, seq_len, hidden_dim)
        logits, output = self.decoder(dec_src, enc_src, enc_output)
        # logits = (bs, seq_len, VOCAB_SIZE) 

        return logits, output

## Model Train

### Create Model

In [79]:
model = Transformer(N, HIDDEN_DIM, NUM_HEAD, INNER_DIM).to(device)
ic.disable() #ic를 통해 debug를 쉽게해주는데 => 이를 비활성화 시킴

### Check Model Structure

In [80]:
from torchsummary import summary
test1 = torch.randint(low = 0, high = 1000, size = (SEQ_LEN,))
test2 = torch.randint(low = 0, high = 1000, size = (SEQ_LEN,))
summary(model, [(SEQ_LEN,), (SEQ_LEN,)], dtypes = [torch.int, torch.int])

Layer (type:depth-idx)                        Output Shape              Param #
├─Encoder: 1-1                                [-1, 60, 256]             --
|    └─Embedding: 2-1                         [-1, 60, 256]             1,280,000
|    └─Embedding: 2-2                         [-1, 60, 256]             25,600
|    └─Dropout: 2-3                           [-1, 60, 256]             --
|    └─Dropout: 2-4                           [-1, 60, 256]             --
|    └─ModuleList: 2                          []                        --
|    |    └─EncoderLayer: 3-1                 [-1, 60, 256]             527,104
|    |    └─EncoderLayer: 3-2                 [-1, 60, 256]             527,104
├─Decoder: 1-2                                [-1, 60, 5000]            --
|    └─Embedding: 2-5                         [-1, 60, 256]             1,280,000
|    └─Dropout: 2-6                           [-1, 60, 256]             --
|    └─ModuleList: 2                          []                   

Layer (type:depth-idx)                        Output Shape              Param #
├─Encoder: 1-1                                [-1, 60, 256]             --
|    └─Embedding: 2-1                         [-1, 60, 256]             1,280,000
|    └─Embedding: 2-2                         [-1, 60, 256]             25,600
|    └─Dropout: 2-3                           [-1, 60, 256]             --
|    └─Dropout: 2-4                           [-1, 60, 256]             --
|    └─ModuleList: 2                          []                        --
|    |    └─EncoderLayer: 3-1                 [-1, 60, 256]             527,104
|    |    └─EncoderLayer: 3-2                 [-1, 60, 256]             527,104
├─Decoder: 1-2                                [-1, 60, 5000]            --
|    └─Embedding: 2-5                         [-1, 60, 256]             1,280,000
|    └─Dropout: 2-6                           [-1, 60, 256]             --
|    └─ModuleList: 2                          []                   

### Weight Init

In [81]:
for param in model.named_parameters():
    if 'weight' in param[0] and 'layerNorm' not in param[0] :
        torch.nn.init.xavier_uniform_(param[1])

### Optimizer

In [82]:
optimizer = torch.optim.Adam(params = model.parameters(), lr = LEARNING_RATE, weight_decay = WEIGHT_DECAY)

### Loss Function

In [83]:
def criterion(logits: torch.tensor, targets: torch.tensor):
    return nn.CrossEntropyLoss(ignore_index=PAD_IDX)(logits.view(-1,VOCAB_SIZE), targets.view(-1))

### Training Function

In [84]:
def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch):
    # train 모드로 변경
    model.train()

    # for the Mixed Precision
    # Pytorch 예제 : https://pytorch.org/docs/stable/notes/amp_examples.html#amp-examples
    if(FP16):
        scaler = amp.GradScaler()

    dataset_size = 0
    running_loss = 0
    running_accuracy = 0
    accuracy = 0

    bar = tqdm(enumerate(dataloader), total=len(dataloader))

    for step, (src, trg_input, trg_output) in bar:
        src = src.to(device)
        trg_input = trg_input.to(device)
        trg_output = trg_output.to(device)

        batch_size = src.shape[0]

        if(FP16):
            with amp.autocast(enabled=True):
                logits, output = model(enc_src=src, dec_src=trg_input)
                loss = criterion(logits, trg_output)

                # loss를 Scale
                # Scaled Grdients를 계산(call)하기 위해 scaled loss를 backward()
                scaler.scale(loss).backward()
                # scaler.step() first unscales the gradients of the optimizer's assigned params.
                # If these gradients do not contain infs or NaNs, optimizer.step() is then called,
                # otherwise, optimizer.step() is skipped.
                scaler.step(optimizer)

                # Updates the scale for next iteration.
                scaler.update()

        else:
            logits, output = model(enc_src=src, dec_src=trg_input)
            loss = criterion(logits, trg_output)

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
            optimizer.step()

        # logits (bs, seq_len, VOCAB_SIZE)
        # trg_output (bs, seq_len)

        # zero the parameter gradients
        optimizer.zero_grad()

        # change learning rate by Scheduler
        if scheduler is not None:
            scheduler.step()

        # loss.item()은 loss를 Python Float으로 반환
        # loss.item()은 batch data의 average loss이므로, sum of loss를 구하기 위해 batch_size를 곱해준다
        running_loss += loss.item() * batch_size
        running_accuracy = np.mean(
            output.view(-1).detach().cpu().numpy() == trg_output.view(-1).detach().cpu().numpy())

        accuracy += running_accuracy

        dataset_size += batch_size
        epoch_loss = running_loss / dataset_size

        bar.set_postfix(
            Epoch=epoch, Train_Loss=epoch_loss, LR=optimizer.param_groups[0]["lr"], accuracy=accuracy / float(
                step+1)
        )

        # break

    accuracy /= len(dataloader)
    # Garbage Collector
    gc.collect()

    return epoch_loss, accuracy

### Validation Function

In [85]:
@torch.no_grad()
def valid_one_epoch(model, dataloader, device, epoch):
    model.eval()

    dataset_size = 0
    running_loss = 0
    accuracy = 0

    bar = tqdm(enumerate(dataloader), total=len(dataloader))

    for step, (src, trg_input, trg_output) in bar:
        src = src.to(device)
        trg_input = trg_input.to(device)
        trg_output = trg_output.to(device)

        batch_size = src.shape[0]

        logits, output = model(enc_src = src, dec_src = trg_input)
        loss = criterion(logits, trg_output)

        running_loss += loss.item() * batch_size
        dataset_size += batch_size

        # 실시간으로 정보를 표시하기 위한 epoch loss
        val_loss = running_loss / dataset_size
        running_accuracy = np.mean(output.view(-1).detach().cpu().numpy() == trg_output.view(-1).detach().cpu().numpy())
        
        accuracy += running_accuracy

        bar.set_postfix(
            Epoch=epoch, Valid_Loss=val_loss, LR=optimizer.param_groups[0]["lr"], accuracy = accuracy / float(step + 1)
        )

        # break

    accuracy /= len(dataloader)

    gc.collect()

    return val_loss, accuracy

In [86]:
def run_training(
    model,
    optimizer,
    scheduler,
    device,
    num_epochs,
    train_dataloader,
    valid_dataloader,
    file_prefix="",
    early_stopping=True,
    early_stopping_step=10,
):
    if torch.cuda.is_available():
        print("[INFO] Using GPU:{}\n".format(torch.cuda.get_device_name()))

    start = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = np.inf
    history = defaultdict(list)
    early_stop_counter = 0

    for epoch in range(1, num_epochs + 1):
        gc.collect()

        train_epoch_loss, train_accuracy = train_one_epoch(
            model,
            optimizer,
            scheduler,
            dataloader=train_dataloader,
            device=device,
            epoch=epoch,
        )

        val_loss, val_accuracy = valid_one_epoch(
            model, valid_dataloader, device=device, epoch=epoch
        )

        # Log metrics
        print(f"Epoch [{epoch}/{num_epochs}], Train Loss: {train_epoch_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")
        print(f"Epoch [{epoch}/{num_epochs}], Valid Loss: {val_loss:.4f}, Valid Accuracy: {val_accuracy:.4f}")

        # Update history
        history['Train Loss'].append(train_epoch_loss)
        history['Train Accuracy'].append(train_accuracy)
        history['Valid Loss'].append(val_loss)
        history['Valid Accuracy'].append(val_accuracy)

        # Save best model
        if val_loss <= best_loss:
            early_stop_counter = 0
            print(f"Validation Loss improved ({best_loss:.4f} ---> {val_loss:.4f})")

            best_loss = val_loss
            best_model_wts = copy.deepcopy(model.state_dict())

            # Save model weights
            model_path = f"{file_prefix}epoch_{epoch}_loss_{best_loss:.4f}.pth"
            torch.save(model.state_dict(), model_path)
            print(f"Model Saved: {model_path}")

        elif early_stopping:
            early_stop_counter += 1
            if early_stop_counter > early_stopping_step:
                break

    end = time.time()
    time_elapsed = end - start
    print(
        "Training complete in {:.0f}h {:.0f}m {:.0f}s".format(
            time_elapsed // 3600,
            (time_elapsed % 3600) // 60,
            (time_elapsed % 3600) % 60,
        )
    )
    print("Best Loss: {:.4f}".format(best_loss))

    # Load best model weights
    model.load_state_dict(best_model_wts)

    return model, history

### Run training

In [87]:
run_training(
    model=model,
    optimizer=optimizer,
    scheduler=torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max=100, eta_min=1e-5),
    device=device,
    num_epochs=50,
    train_dataloader= train_dataloader,
    valid_dataloader = valid_dataloader,
    file_prefix="./model/",
    early_stopping=True,
    early_stopping_step=10,
)

c:\Users\thoma\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\amp\grad_scaler.py:131: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
  0%|          | 0/389 [00:00<?, ?it/s]c:\Users\thoma\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\amp\autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
100%|██████████| 98/98 [00:38<00:00,  2.57it/s, Epoch=1, LR=9.73e-5, Valid_Loss=7.06, accuracy=0.0577]


Epoch [1/50], Train Loss: 7.4007, Train Accuracy: 0.0281
Epoch [1/50], Valid Loss: 7.0619, Valid Accuracy: 0.0577
Validation Loss improved (inf ---> 7.0619)
Model Saved: ./model/epoch_1_loss_7.0619.pth


100%|██████████| 98/98 [00:41<00:00,  2.35it/s, Epoch=2, LR=8.97e-5, Valid_Loss=6.46, accuracy=0.0707]


Epoch [2/50], Train Loss: 6.7609, Train Accuracy: 0.0657
Epoch [2/50], Valid Loss: 6.4610, Valid Accuracy: 0.0707
Validation Loss improved (7.0619 ---> 6.4610)
Model Saved: ./model/epoch_2_loss_6.4610.pth


100%|██████████| 98/98 [00:35<00:00,  2.74it/s, Epoch=3, LR=7.79e-5, Valid_Loss=6.04, accuracy=0.0804]


...

100%|██████████| 98/98 [00:39<00:00,  2.47it/s, Epoch=49, LR=3.98e-5, Valid_Loss=3.9, accuracy=0.151] 


Epoch [49/50], Train Loss: 3.4426, Train Accuracy: 0.1653
Epoch [49/50], Valid Loss: 3.8974, Valid Accuracy: 0.1509
Validation Loss improved (3.9047 ---> 3.8974)
Model Saved: ./model/epoch_49_loss_3.8974.pth


100%|██████████| 98/98 [00:39<00:00,  2.50it/s, Epoch=50, LR=5.5e-5, Valid_Loss=3.89, accuracy=0.151]


Epoch [50/50], Train Loss: 3.4168, Train Accuracy: 0.1667
Epoch [50/50], Valid Loss: 3.8889, Valid Accuracy: 0.1513
Validation Loss improved (3.8974 ---> 3.8889)
Model Saved: ./model/epoch_50_loss_3.8889.pth
Training complete in 5h 5m 34s
Best Loss: 3.8889


(Transformer(
   (encoder): Encoder(
     (embedding): Embedding(5000, 256, padding_idx=0)
     (pos_embedding): Embedding(100, 256)
     (enc_layers): ModuleList(
       (0-1): 2 x EncoderLayer(
         (multiheadattention): Multiheadattention(
           (fcQ): Linear(in_features=256, out_features=256, bias=True)
           (fcK): Linear(in_features=256, out_features=256, bias=True)
           (fcV): Linear(in_features=256, out_features=256, bias=True)
           (fcOut): Linear(in_features=256, out_features=256, bias=True)
           (dropout): Dropout(p=0.1, inplace=False)
         )
         (ffn): FFN(
           (fc1): Linear(in_features=256, out_features=512, bias=True)
           (fc2): Linear(in_features=512, out_features=256, bias=True)
           (relu): ReLU()
           (dropout): Dropout(p=0.1, inplace=False)
         )
         (layerNorm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
         (layerNorm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True

In [88]:
torch.save(model.state_dict(), 'final.bin')

In [93]:
def predict(src_sentence):
    # Prepare Sample Sentence
    dec_sentence = ''

    enc_src = sp_src.EncodeAsIds(src_sentence)
    dec_src = []
    dec_src = np.insert(dec_src, 0, sp_trg.bos_id())
    # dec_src = ko_encode(dec_sentence)

    enc_src = torch.Tensor(enc_src).view(1, -1).int().to(device)
    dec_src = torch.Tensor(dec_src).view(1, -1).int().to(device)
    # enc_src : (1,seq_len)
    # dec_src : (1,seq_len)

    last_token = None
    last_token_idx = 0

    while(True):

        # dec_src에 dec_output의 last token을 추가합니다
        enc_output = model.encoder(enc_src)
        # enc_output : (1,seq_len, hidden_dim)

        dec_logits, dec_output = model.decoder(
            input=dec_src, enc_src=enc_src, enc_output=enc_output
        )
        # dec_output : (1,seq_len)
        # dec_logits : (1, seq_len, VOCAB_SIZE)

        last_token = dec_output[:, last_token_idx].item()
        last_token = torch.Tensor([last_token]).view(-1, 1).int()

        # last_token : (1, 1)
        dec_src = torch.cat((dec_src, last_token), dim=-1)

        last_token_idx = last_token_idx + 1

        # print(dec_src)
        # print(sp_trg.Decode(dec_src.tolist()))
        # print(last_token.item())
        if last_token.item() is EOS_IDX:
            break

    # ic(dec_src.tolist())
    return sp_trg.Decode(dec_src.tolist())

In [94]:
# Prepare 10 Sample Sentence
indices = np.random.choice(len(data['en']), 10, replace=False)
sentences = data['en'][indices].to_list()
answers = data['ko'][indices].to_list()

for idx in range(len(sentences)):
    sentence = sentences[idx]
    print(f'en = {sentence}')
    print(f'answer = {answers[idx]}')
    print(f'ko = {predict(sentence)}')

en =  I would state my case before him and fill my mouth with arguments.
answer =  그분 앞에서 내 사정을 아뢰련만, 내가 정당함을 입이 닳도록 변론하련만.
ko = ['내가 내 말을 내 앞에 무릎을 꿇고, 내 입술로 내 입술로 내 입술로 내 입술로 내 입술로 나아오.']
en =  Joseph threw himself upon his father and wept over him and kissed him.
answer =  요셉이 아버지의 얼굴에 엎드려서, 울며 입을 맞추고,
ko = ['요셉이 자기 아버지에게로 달려가서, 그에게 입을 맞추었다.']
en =  It still remains that some will enter that rest, and those who formerly had the gospel preached to them did not go in, because of their disobedience.
answer =  그러나 어떤 사람들에게는 안식에 들어갈 기회가 아직 남아 있습니다. 그런데 기쁜 소식을 먼저 들은 사람들은 순종하지 않았으므로, 들어갈 수 없었습니다.
ko = ['그러나 그 때에 그들은 여러 가지로 내려가지 못할 것이다. 그들은 복음을 전체가 아니라, 복음을 전투하는 사람들을 시켜서, 이방 사람들에게로 내려왔습니다.']
en =  But if your heart turns away and you are not obedient, and if you are drawn away to bow down to other gods and worship them,
answer =  그러나 너희가 마음을 돌려서 순종하지 않고, 빗나가서 다른 신들에게 절을 하고 섬기면,
ko = ['그러나 너희는, 너희의 마음을 다하여 주 너희의 하나님을 섬기며, 너희의 신상을 만들어 버리고, 너희의 신상을 만들어 버리고,']
en =  Only be careful, and watch yo